In [59]:
#The lower the score, the more relevant the keyword is.

In [1]:
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS #import commen list of stopword
import en_core_web_sm
import string
from tqdm import tqdm
tqdm.pandas()
import time
import yake 

In [2]:
%%time 
start = time.time()
chunks = pd.read_json('data/arxiv-metadata-oai-snapshot.json', lines=True, chunksize = 100000)
print('json loaded to chunks')
df = pd.DataFrame()
for idx, chunk in enumerate(chunks):
    print(idx + 1)
    df = pd.concat([df,chunk[['id', 'categories','abstract']]])


json loaded to chunks
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
CPU times: user 1min 48s, sys: 3.68 s, total: 1min 51s
Wall time: 1min 52s


In [3]:
df = df.sample(frac = .001)

In [4]:
%%time
df = df[~df['categories'].str.contains(r'\s')]
df.abstract = df.abstract.astype(str)

CPU times: user 6.14 ms, sys: 997 µs, total: 7.13 ms
Wall time: 6.43 ms


In [6]:
kw_extractor = yake.KeywordExtractor(top=10)

In [111]:
%%time
df['keywords'] = df.abstract.progress_apply(lambda x: kw_extractor.extract_keywords(x.lower()))

  1%|▎                                                            | 6467/1270698 [02:34<8:24:35, 41.76it/s]


KeyboardInterrupt: 

In [112]:
%%time
keywords_df = df.groupby('categories')['keywords'].agg(lambda x: sum(x, [])).reset_index()
keywords_df = keywords_df.explode('keywords').reset_index(drop = True)
keywords_df[['keyword','score']] = keywords_df.keywords.apply(pd.Series)
keywords_df = keywords_df.drop(columns = 'keywords')
result_df = keywords_df.groupby('categories', group_keys=False).apply(lambda group: group.nsmallest(10, 'score'))

KeyError: 'Column not found: keywords'

In [113]:
%%time
result_df.to_csv('data/masterYake_wo_agg.csv', index = False)

CPU times: user 3.09 ms, sys: 27 µs, total: 3.12 ms
Wall time: 2.56 ms
